In [1]:
import tstrain
import tsclust
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [27]:
# Training data - no crop labels
td = pd.read_csv('/home/ec2-user/training_data_large.csv')

/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
td = td.drop(['Unnamed: 0'], axis=1)

In [30]:
# Weird unexpexted strings
td.loc[td['label'] == '0', 'label'] = 0
td.loc[td['label'] == '1', 'label'] = 1
td.loc[td['label'] == '2', 'label'] = 2
td.loc[td['label'] == '3', 'label'] = 3
td.loc[td['label'] == '4', 'label'] = 4

In [31]:
td.label.unique()

array(['veg', 'water', 4, 2, 0, 1, 3, 'urban'], dtype=object)

In [32]:
# Group clusters 2 and 4 as "maize"; group clusters 0 and 1 as "crop_2"; leave cluster 3 alone, call it "crop_3"
td.loc[td['label'] == 2, 'label'] = "maize"
td.loc[td['label'] == 4, 'label'] = "maize"
td.loc[td['label'] == 0, 'label'] = "crop_2"
td.loc[td['label'] == 1, 'label'] = "crop_2"
td.loc[td['label'] == 3, 'label'] = "crop_3"

In [33]:
td.label.unique()

array(['veg', 'water', 'maize', 'crop_2', 'crop_3', 'urban'], dtype=object)

In [35]:
# Use first 6 dates in time series (Nov 16, 2016 through May 25, 2017)
dates = td.date.unique()
datesub = dates[0:6]
trainsub = td[td['date'].isin(datesub)]

In [37]:
# Fit a LSTM recurrent neural network. In this 'toy' example, a total of 25,000 samples are used to fit a model.
# including 10,000 from the clustered "cropped" class, and 5,000 from each of the "water", "urban" and
# "vegetation" classes. The bands (features) include red, blue, green, and nir. Y labels are numerically
# encoded, and converted to "one-hot" vectors.

# Format training data into correct 3D array of shape (n_samples, n_timesetps, n_features) required to fit a
# Keras LSTM model. N_features corresponds to number of bands included in training data

class_codes, x, y = tstrain.format_training_data(trainsub)

/home/ec2-user/temporal-crop-classification/cropclass/tstrain.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  training_data['label'] = training_data['label'].astype('category')
/home/ec2-user/temporal-crop-classification/cropclass/tstrain.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  training_data['label'] = training_data['label'].cat.codes.astype('str').astype('int')


In [38]:
# Split training and test data
x_train, x_test, y_train, y_test = tstrain.split_train_test(x, y, seed=0)

In [39]:
# Standardize features
mu, sd, x_train_norm, x_test_norm = tstrain.standardize_features(x_train, x_test)

In [41]:
# Train LSTM model
n_timesteps = len(trainsub['date'].unique())
n_features = len(trainsub['feature'].unique())

model = Sequential()
model.add(LSTM(32, activation='relu', return_sequences=True, input_shape=(n_timesteps, n_features)))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(32))
model.add(Dense(activation='softmax', units=y.shape[1]))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(x_train_norm, y_train, epochs=50, batch_size=32, verbose=2)

Epoch 1/50
 - 24s - loss: 0.5063 - categorical_accuracy: 0.8042
Epoch 2/50
 - 21s - loss: 0.2836 - categorical_accuracy: 0.8922
Epoch 3/50
 - 21s - loss: 0.2301 - categorical_accuracy: 0.9162
Epoch 4/50
 - 21s - loss: 0.2019 - categorical_accuracy: 0.9273
Epoch 5/50
 - 21s - loss: 0.1786 - categorical_accuracy: 0.9336
Epoch 6/50
 - 21s - loss: 0.1687 - categorical_accuracy: 0.9373
Epoch 7/50
 - 21s - loss: 0.1569 - categorical_accuracy: 0.9427
Epoch 8/50
 - 21s - loss: 0.1427 - categorical_accuracy: 0.9476
Epoch 9/50
 - 21s - loss: 0.1371 - categorical_accuracy: 0.9486
Epoch 10/50
 - 21s - loss: 0.1266 - categorical_accuracy: 0.9518
Epoch 11/50
 - 21s - loss: 0.1167 - categorical_accuracy: 0.9556
Epoch 12/50
 - 21s - loss: 0.1137 - categorical_accuracy: 0.9585
Epoch 13/50
 - 21s - loss: 0.1070 - categorical_accuracy: 0.9606
Epoch 14/50
 - 21s - loss: 0.1041 - categorical_accuracy: 0.9615
Epoch 15/50
 - 21s - loss: 0.1014 - categorical_accuracy: 0.9632
Epoch 16/50
 - 21s - loss: 0.0970 

In [42]:
# Model accuracy
_, accuracy = model.evaluate(x_test_norm, y_test, batch_size=32)
accuracy

7191/7191 [==============================] - 2s 214us/step


0.9712140175219024

In [43]:
# Confusion matrix
tstrain.conf_mat(x_test_norm, y_test, model, class_codes)

,crop_2,crop_3,maize,urban,veg,water,recall
crop_2,793,3,50,4,3,0,0.929660
crop_3,11,58,2,0,0,0,0.816901
maize,47,5,985,0,15,0,0.936312
urban,7,0,3,1179,1,0,0.990756
veg,33,1,23,0,1945,0,0.971528
water,0,0,0,0,0,2023,1.000000


In [44]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/ec2-user/model_labeled.json", "w") as json_file:
    json_file.write(model_json)

In [45]:
# serialize weights to HDF5
model.save_weights("/home/ec2-user/model_labeled.h5")
print("Saved model to disk")

Saved model to disk
